In [ ]:
import pandas as pd
import ast
import re
import numpy as np

asset_list = [
    "SPY", "QQQ", "DIA",
    "EWJ", "EEM", "VGK",
    "TLT", "IEF", "BND",
    "GLD", "SLV", "USO",
    "EURUSD=X", "JPY=X", "AUDUSD=X",
    "BTC-USD", "ETH-USD"
]

file_path = "backtest_by_strategy_{strategy_name}.xlsx"

def parse_perf(x):
    if pd.isna(x):
        return None

    if isinstance(x, dict):
        return x

    if not isinstance(x, str):
        raise TypeError(f"Unexpected type: {type(x)}")

    s = x

    # 1. Remove numpy wrappers
    s = re.sub(r'np\.float64\(([^)]+)\)', r'\1', s)

    # 2. Replace invalid literals BEFORE parsing
    s = re.sub(r'\bNaN\b|\bnan\b', 'None', s)
    s = re.sub(r'\binf\b|\b-inf\b', 'None', s)

    # 3. Remove unary minus applied to None (all forms)
    s = re.sub(r'-\s*\(?\s*None\s*\)?', 'None', s)

    try:
        d = ast.literal_eval(s)
    except Exception:
        print("FAILED STRING ↓↓↓")
        print(s)
        raise

    # 4. Final numeric sanitation (post-parse)
    for k, v in d.items():
        if v is None:
            continue
        if isinstance(v, float) and (np.isnan(v) or np.isinf(v)):
            d[k] = None

    return d

def extract_ssr_ftest_values(granger_str):
    if not isinstance(granger_str, str):
        return np.empty((0, 4))  # no tests found

    matches = re.findall(
        r"'ssr_ftest'\s*:\s*\(\s*"
        r"np\.float64\(([\d.eE+-]+)\)\s*,\s*"
        r"np\.float64\(([\d.eE+-]+)\)\s*,\s*"
        r"np\.float64\(([\d.eE+-]+)\)\s*,\s*"
        r"np\.int64\(([\d.eE+-]+)\)\s*"
        r"\)",
        granger_str
    )

    return np.array(matches, dtype=float)

def parse_beta(x):
    if pd.isna(x):
        return None
    
    if isinstance(x, dict):
        return x
    
    if isinstance(x, str):
        x = x.replace("np.float64(", "").replace(")", "")
        try:
            return ast.literal_eval(x)
        except Exception:
            return None
    
    return None

def avg_pvalue_from_ssr(ssr):
    if ssr.size == 0:
        return np.nan
    return ssr[:, 1].mean()   # column 1 = p-value



In [ ]:
data = {}

for asset in asset_list:
    sheet = f"{asset}_{strategy_name}"
    df = pd.read_excel(file_path, sheet_name=sheet)
    df["perf_dict"] = df["perf_metric_0"].apply(parse_perf)
    metrics_df = pd.json_normalize(df["perf_dict"])
    df = pd.concat([df, metrics_df], axis=1)
    df["factor_regression_beta_dict"] = df["factor_regression_beta"].apply(parse_beta)
    betas = pd.json_normalize(df["factor_regression_beta_dict"])
    df = pd.concat([df, betas], axis=1)
    df["binom_pvalue"] = (
        df["accuracy_binom_p"]
        .str.extract(r"pvalue=([0-9.eE+-]+)")
        .astype(float)
    )
    df["avg_granger1_pvalue"] = (
        df["signal_causes_returns"]
        .apply(lambda x: avg_pvalue_from_ssr(extract_ssr_ftest_values(x)))
    )

    df["avg_granger2_pvalue"] = (
        df["returns_cause_signal"]
        .apply(lambda x: avg_pvalue_from_ssr(extract_ssr_ftest_values(x)))
    )
    data[asset] = df

In [ ]:
performance_metrics = []
valid_values = []

for asset in asset_list:
    df = data[asset]
    df[["CAGR", "Max Drawdown", "jensens_alpha"]] = df[["CAGR", "Max Drawdown", "jensens_alpha"]].fillna(0)
    nan_ratio = df.isna().sum().sum() / (df.shape[0] * df.shape[1])
    metrics = {
        "asset": asset,
        "mean_cagr": df["CAGR"].mean(),
        "median_max_drawdown": df["Max Drawdown"].median(),
        "mean_sharpe": df["Sharpe Ratio"].mean(),
        "mean_sortino": df["Sortino Ratio"].mean(),
        "mean_profit_factor": df["Profit Factor"].mean(),
        "mean_30d_rolling_sharpe": df["30 days Rolling Sharpe"].mean(),
        "mean_jensens_alpha": df["jensens_alpha"].mean(),
        "nan_ratio": nan_ratio
    }

    


    performance_metrics.append(metrics)

performance_df = pd.DataFrame(performance_metrics)

In [ ]:
performance_df

In [ ]:
threshold = 0.05
significance_metrics = []
significance_counts = []

for asset in asset_list:
    df = data[asset]

    count_ttest_parameters_below_threshold = (df["paired_t_p"] < threshold).sum()
    count_valid_ttest_values = df["paired_t_p"].notna().sum()
    percentage_ttest_parameters_below_threshold = (count_ttest_parameters_below_threshold / count_valid_ttest_values) * 100

    count_wilcoxon_parameters_below_threshold = (df["wilcoxon_p"] < threshold).sum()
    count_valid_wilcoxon_values = df["wilcoxon_p"].notna().sum()
    percentage_wilcoxon_parameters_below_threshold = (count_wilcoxon_parameters_below_threshold / count_valid_wilcoxon_values) * 100

    count_newey_west_parameters_below_threshold = (df["nw_alpha_pvalue"] < threshold).sum()
    count_valid_newey_west_values = df["nw_alpha_pvalue"].notna().sum()
    percentage_newey_west_parameters_below_threshold = (count_newey_west_parameters_below_threshold / count_valid_newey_west_values) * 100

    count_ledoit_wolf_parameters_below_threshold = (df["lw_sharpe_pvalue"] < threshold).sum()
    count_valid_ledoit_wolf_values = df["lw_sharpe_pvalue"].notna().sum()
    percentage_ledoit_wolf_parameters_below_threshold = (count_ledoit_wolf_parameters_below_threshold / count_valid_ledoit_wolf_values) * 100

    count_bootstrap_significance_parameters_below_threshold = (df["bootstrap_pvalue"] < threshold).sum()
    count_valid_bootstrap_values = df["bootstrap_pvalue"].notna().sum()
    percentage_bootstrap_parameters_below_threshold = (count_bootstrap_significance_parameters_below_threshold / count_valid_bootstrap_values) * 100

    count_directional_accuracy_significant_positive_parameters = ((df["accuracy"] > 0.5) & (df["binom_pvalue"] < threshold)).sum()
    count_directional_accuracy_significant_negative_parameters = ((df["accuracy"] < 0.5) & (df["binom_pvalue"] < threshold)).sum()
    count_directional_accuracy_insignificant_parameters = (df["binom_pvalue"] >= threshold).sum()
    count_valid_accuracy_values = df["accuracy"].notna().sum()
    count_valid_binom_pvalue_values = df["binom_pvalue"].notna().sum()
    count_valid_dirrectional_accuracy_values = count_valid_accuracy_values + count_valid_binom_pvalue_values
    percentage_directional_accuracy_significant_positive_parameters = (count_directional_accuracy_significant_positive_parameters / count_valid_dirrectional_accuracy_values) * 100
    percentage_directional_accuracy_significant_negative_parameters = (count_directional_accuracy_significant_negative_parameters / count_valid_dirrectional_accuracy_values) * 100
    percentage_directional_accuracy_insignificant_parameters = (count_directional_accuracy_insignificant_parameters / count_valid_dirrectional_accuracy_values) * 100


    count_granger1_parameters_below_threshold = (df["avg_granger1_pvalue"] < threshold).sum()
    count_valid_granger1_values = df["avg_granger1_pvalue"].notna().sum()
    percentage_granger1_parameters_below_threshold = (count_granger1_parameters_below_threshold / count_valid_granger1_values) * 100


    count_granger2_parameters_below_threshold = (df["avg_granger2_pvalue"] < threshold).sum()
    count_valid_granger2_values = df["avg_granger2_pvalue"].notna().sum()
    percentage_granger2_parameters_below_threshold = (count_granger2_parameters_below_threshold / count_valid_granger2_values) * 100
    

    count_random_permutation_parameters_below_threshold = (df["permutation_pvalue"] < threshold).sum()
    count_valid_random_permutation_values = df["permutation_pvalue"].notna().sum()
    percentage_random_permutation_parameters_below_threshold = (count_random_permutation_parameters_below_threshold / count_valid_random_permutation_values) * 100


    count_innovation_test_parameters_below_threshold = (df["innovation_test_pvalue"] < threshold).sum()
    count_valid_innovation_test_values = df["innovation_test_pvalue"].notna().sum()
    percentage_innovation_test_parameters_below_threshold = (count_innovation_test_parameters_below_threshold / count_valid_innovation_test_values) * 100


    count_adf_test_parameters_below_threshold = (df["adf_p"] < threshold).sum()
    count_valid_adf_values = df["adf_p"].notna().sum()
    percentage_adf_test_parameters_below_threshold = (count_adf_test_parameters_below_threshold / count_valid_adf_values) * 100

    count_kpss_test_parameters_below_threshold = (df["kpss_p"] < threshold).sum()
    count_valid_kpss_values = df["kpss_p"].notna().sum()
    percentage_kpss_test_parameters_below_threshold = (count_kpss_test_parameters_below_threshold / count_valid_kpss_values) * 100

    mean_factor_reg_alpha_across_parameters = df["factor_regression_alpha"].mean()
    median_factor_reg_alpha_across_parameters = df["factor_regression_alpha"].median()
    count_factor_reg_alpha_parameters_below_threshold = (df["factor_regression_alpha_pvalue"] < threshold).sum()
    count_valid_factor_reg_alpha_values = df["factor_regression_alpha"].notna().sum()
    count_valid_factor_reg_alpha_pvalues_values = df["factor_regression_alpha_pvalue"].notna().sum()
    percentage_factor_reg_alpha_parameters_below_threshold = (count_factor_reg_alpha_parameters_below_threshold / count_valid_factor_reg_alpha_pvalues_values) * 100


    metrics ={
        "asset": asset,
        "percentage_ttest_below_0.05": percentage_ttest_parameters_below_threshold,
        "percentage_wilcoxon_below_0.05": percentage_wilcoxon_parameters_below_threshold,
        "percentage_newey_west_below_0.05": percentage_newey_west_parameters_below_threshold,
        "percentage_ledoit_wolf_below_0.05": percentage_ledoit_wolf_parameters_below_threshold,
        "percentage_bootstrap_below_0.05": percentage_bootstrap_parameters_below_threshold,
        "percentage_directional_accuracy_significant_positive": percentage_directional_accuracy_significant_positive_parameters,
        "percentage_directional_accuracy_significant_negative": percentage_directional_accuracy_significant_negative_parameters,
        "percentage_directional_accuracy_insignificant": percentage_directional_accuracy_insignificant_parameters,
        "percentage_granger1_below_0.05": percentage_granger1_parameters_below_threshold,
        "percentage_granger2_below_0.05": percentage_granger2_parameters_below_threshold,
        "percentage_random_permutation_below_0.05": percentage_random_permutation_parameters_below_threshold,
        "percentage_innovation_test_below_0.05": percentage_innovation_test_parameters_below_threshold,
        "percentage_adf_test_below_0.05": percentage_adf_test_parameters_below_threshold,
        "percentage_kpss_test_below_0.05": percentage_kpss_test_parameters_below_threshold,
        "mean_factor_reg_alpha_across_parameters": mean_factor_reg_alpha_across_parameters,
        "median_factor_reg_alpha_across_parameters": median_factor_reg_alpha_across_parameters,
        "percentage_factor_reg_alpha_across_parameters": percentage_factor_reg_alpha_parameters_below_threshold
    }

    counts ={
        "asset": asset,
        "count_ttest_below_0.05": count_ttest_parameters_below_threshold,
        "count_ttest_valid_values": count_valid_ttest_values,
        "count_wilcoxon_below_0.05": count_wilcoxon_parameters_below_threshold,
        "count_wilcoxon_valid_values": count_valid_wilcoxon_values,
        "count_newey_west_below_0.05": count_newey_west_parameters_below_threshold,
        "count_newey_west_valid_values": count_valid_newey_west_values,
        "count_ledoit_wolf_below_0.05": count_ledoit_wolf_parameters_below_threshold,
        "count_ledoit_wolf_valid_values": count_valid_ledoit_wolf_values,
        "count_bootstrap_below_0.05": count_bootstrap_significance_parameters_below_threshold,
        "count_bootstrap_valid_values": count_valid_bootstrap_values,
        "count_directional_accuracy_significant_positive": count_directional_accuracy_significant_positive_parameters,
        "count_directional_accuracy_significant_negative": count_directional_accuracy_significant_negative_parameters,
        "count_directional_accuracy_insignificant": count_directional_accuracy_insignificant_parameters,
        "count_granger1_below_0.05": count_granger1_parameters_below_threshold,
        "count_granger1_valid_values": count_valid_granger1_values,
        "count_granger2_below_0.05": count_granger2_parameters_below_threshold,
        "count_granger2_valid_values": count_valid_granger2_values,
        "count_random_permutation_below_0.05": count_random_permutation_parameters_below_threshold,
        "count_random_permutation_valid_values": count_valid_random_permutation_values,
        "count_innovation_test_below_0.05": count_innovation_test_parameters_below_threshold,
        "count_innovation_test_valid_values": count_valid_innovation_test_values,
        "count_adf_test_below_0.05": count_adf_test_parameters_below_threshold,
        "count_adf_test_valid_values": count_valid_adf_values,
        "count_kpss_test_below_0.05": count_kpss_test_parameters_below_threshold,
        "count_kpss_test_valid_values": count_valid_kpss_values,
        "count_factor_reg_alpha_below_0.05": count_factor_reg_alpha_parameters_below_threshold,
        "count_valid_factor_reg_alpha_values": count_valid_factor_reg_alpha_values,
        "count_valid_factor_reg_alpha_pvalues_values": count_valid_factor_reg_alpha_pvalues_values
    }

    significance_metrics.append(metrics)
    significance_counts.append(counts)

significance_df = pd.DataFrame(significance_metrics)
significance_counts_df = pd.DataFrame(significance_counts)

In [ ]:
significance_df

In [ ]:
significance_counts_df

In [ ]:
high_vol_df = []

for asset in asset_list:
    df = data[asset]
    mean_hvol_strategy_total_returns = df["hvol_strategy total return"].mean()
    median_hvol_strategy_total_returns = df["hvol_strategy total return"].median()
    mean_hvol_strategy_annualized_return = df["hvol_ann. strategy return"].mean()
    median_hvol_strategy_annualized_return = df["hvol_ann. strategy return"].median()
    mean_hvol_strategy_volatility = df["hvol_ann. strategy volatility"].mean()
    median_hvol_strategy_volatility = df["hvol_ann. strategy volatility"].median()
    mean_hvol_strategy_sharpe = df["hvol_strategy sharpe ratio"].mean()
    median_hvol_strategy_sharpe = df["hvol_strategy sharpe ratio"].median()
    mean_hvol_strategy_winrate = df["hvol_strategy winrate"].mean()
    median_hvol_strategy_winrate = df["hvol_strategy winrate"].median()

    metrics = {
        "asset": asset,
        "mean_hvol_strategy_total_returns": mean_hvol_strategy_total_returns,
        "median_hvol_strategy_total_returns": median_hvol_strategy_total_returns,
        "mean_hvol_strategy_annualized_return": mean_hvol_strategy_annualized_return,
        "median_hvol_strategy_annualized_return": median_hvol_strategy_annualized_return,
        "mean_hvol_strategy_volatility": mean_hvol_strategy_volatility,
        "median_hvol_strategy_volatility": median_hvol_strategy_volatility,
        "mean_hvol_strategy_sharpe": mean_hvol_strategy_sharpe,
        "median_hvol_strategy_sharpe": median_hvol_strategy_sharpe,
        "mean_hvol_strategy_winrate": mean_hvol_strategy_winrate,
        "median_hvol_strategy_winrate": median_hvol_strategy_winrate
    }
    high_vol_df.append(metrics)

high_vol_df = pd.DataFrame(high_vol_df)

In [ ]:
high_vol_df

In [ ]:
low_vol_df = []

for asset in asset_list:
    df = data[asset]
    mean_lvol_strategy_total_returns = df["lvol_strategy total return"].mean()
    median_lvol_strategy_total_returns = df["lvol_strategy total return"].median()
    mean_lvol_strategy_annualized_return = df["lvol_ann. strategy return"].mean()
    median_lvol_strategy_annualized_return = df["lvol_ann. strategy return"].median()
    mean_lvol_strategy_volatility = df["lvol_ann. strategy volatility"].mean()
    median_lvol_strategy_volatility = df["lvol_ann. strategy volatility"].median()
    mean_lvol_strategy_sharpe = df["lvol_strategy sharpe ratio"].mean()
    median_lvol_strategy_sharpe = df["lvol_strategy sharpe ratio"].median()
    mean_lvol_strategy_winrate = df["lvol_strategy winrate"].mean()
    median_lvol_strategy_winrate = df["lvol_strategy winrate"].median()

    metrics = {
        "asset": asset,
        "mean_lvol_strategy_total_returns": mean_lvol_strategy_total_returns,
        "median_lvol_strategy_total_returns": median_lvol_strategy_total_returns,
        "mean_lvol_strategy_annualized_return": mean_lvol_strategy_annualized_return,
        "median_lvol_strategy_annualized_return": median_lvol_strategy_annualized_return,
        "mean_lvol_strategy_volatility": mean_lvol_strategy_volatility,
        "median_lvol_strategy_volatility": median_lvol_strategy_volatility,
        "mean_lvol_strategy_sharpe": mean_lvol_strategy_sharpe,
        "median_lvol_strategy_sharpe": median_lvol_strategy_sharpe,
        "mean_lvol_strategy_winrate": mean_lvol_strategy_winrate,
        "median_lvol_strategy_winrate": median_lvol_strategy_winrate
    }
    low_vol_df.append(metrics)

low_vol_df = pd.DataFrame(low_vol_df)
low_vol_df

In [ ]:
spy

In [ ]:
high_vol_buy_n_hold_df = []

for asset in asset_list:
    df = data[asset]
    mean_hvol_buy_n_hold_total_returns = df["hvol_ann. buy n hold return"].mean()
    median_hvol_buy_n_hold_total_returns = df["hvol_ann. buy n hold return"].median()
    mean_hvol_buy_n_hold_annualized_return = df["hvol_ann. buy n hold return"].mean()
    median_hvol_buy_n_hold_annualized_return = df["hvol_ann. buy n hold return"].median()
    mean_hvol_buy_n_hold_volatility = df["hvol_ann. buy n hold volatility"].mean()
    median_hvol_buy_n_hold_volatility = df["hvol_ann. buy n hold volatility"].median()
    mean_hvol_buy_n_hold_sharpe = df["hvol_buy n hold sharpe ratio"].mean()
    median_hvol_buy_n_hold_sharpe = df["hvol_buy n hold sharpe ratio"].median()

    metrics = {
        "asset": asset,
        "mean_hvol_buy_n_hold_total_returns": mean_hvol_buy_n_hold_total_returns,
        "median_hvol_buy_n_hold_total_returns": median_hvol_buy_n_hold_total_returns,
        "mean_hvol_buy_n_hold_annualized_return": mean_hvol_buy_n_hold_annualized_return,
        "median_hvol_buy_n_hold_annualized_return": median_hvol_buy_n_hold_annualized_return,
        "mean_hvol_buy_n_hold_volatility": mean_hvol_buy_n_hold_volatility,
        "median_hvol_buy_n_hold_volatility": median_hvol_buy_n_hold_volatility,
        "mean_hvol_buy_n_hold_sharpe": mean_hvol_buy_n_hold_sharpe,
        "median_hvol_buy_n_hold_sharpe": median_hvol_buy_n_hold_sharpe,
    }
    high_vol_buy_n_hold_df.append(metrics)

high_vol_buy_n_hold_df = pd.DataFrame(high_vol_buy_n_hold_df)
high_vol_buy_n_hold_df

In [ ]:
low_vol_buy_n_hold_df = []

for asset in asset_list:
    df = data[asset]
    mean_lvol_buy_n_hold_total_returns = df["lvol_ann. buy n hold return"].mean()
    median_lvol_buy_n_hold_total_returns = df["lvol_ann. buy n hold return"].median()
    mean_lvol_buy_n_hold_annualized_return = df["lvol_ann. buy n hold return"].mean()
    median_lvol_buy_n_hold_annualized_return = df["lvol_ann. buy n hold return"].median()
    mean_lvol_buy_n_hold_volatility = df["lvol_ann. buy n hold volatility"].mean()
    median_lvol_buy_n_hold_volatility = df["lvol_ann. buy n hold volatility"].median()
    mean_lvol_buy_n_hold_sharpe = df["lvol_buy n hold sharpe ratio"].mean()
    median_lvol_buy_n_hold_sharpe = df["lvol_buy n hold sharpe ratio"].median()

    metrics = {
        "asset": asset,
        "mean_lvol_buy_n_hold_total_returns": mean_lvol_buy_n_hold_total_returns,
        "median_lvol_buy_n_hold_total_returns": median_lvol_buy_n_hold_total_returns,
        "mean_lvol_buy_n_hold_annualized_return": mean_lvol_buy_n_hold_annualized_return,
        "median_lvol_buy_n_hold_annualized_return": median_lvol_buy_n_hold_annualized_return,
        "mean_lvol_buy_n_hold_volatility": mean_lvol_buy_n_hold_volatility,
        "median_lvol_buy_n_hold_volatility": median_lvol_buy_n_hold_volatility,
        "mean_lvol_buy_n_hold_sharpe": mean_lvol_buy_n_hold_sharpe,
        "median_lvol_buy_n_hold_sharpe": median_lvol_buy_n_hold_sharpe,
    }
    low_vol_buy_n_hold_df.append(metrics)

low_vol_buy_n_hold_df = pd.DataFrame(low_vol_buy_n_hold_df)
low_vol_buy_n_hold_df

In [ ]:
recession_df = []

for asset in asset_list:
    df = data[asset]
    mean_rec_strategy_total_returns = df["rec_strategy total return"].mean()
    median_rec_strategy_total_returns = df["rec_strategy total return"].median()
    mean_rec_strategy_annualized_return = df["rec_ann. strategy return"].mean()
    median_rec_strategy_annualized_return = df["rec_ann. strategy return"].median()
    mean_rec_strategy_volatility = df["rec_ann. strategy volatility"].mean()
    median_rec_strategy_volatility = df["rec_ann. strategy volatility"].median()
    mean_rec_strategy_sharpe = df["rec_strategy sharpe ratio"].mean()
    median_rec_strategy_sharpe = df["rec_strategy sharpe ratio"].median()
    mean_rec_strategy_winrate = df["rec_strategy winrate"].mean()
    median_rec_strategy_winrate = df["rec_strategy winrate"].median()

    metrics = {
        "asset": asset,
        "mean_rec_strategy_total_returns": mean_rec_strategy_total_returns,
        "median_rec_strategy_total_returns": median_rec_strategy_total_returns,
        "mean_rec_strategy_annualized_return": mean_rec_strategy_annualized_return,
        "median_rec_strategy_annualized_return": median_rec_strategy_annualized_return,
        "mean_rec_strategy_volatility": mean_rec_strategy_volatility,
        "median_rec_strategy_volatility": median_rec_strategy_volatility,
        "mean_rec_strategy_sharpe": mean_rec_strategy_sharpe,
        "median_rec_strategy_sharpe": median_rec_strategy_sharpe,
        "mean_rec_strategy_winrate": mean_rec_strategy_winrate,
        "median_rec_strategy_winrate": median_rec_strategy_winrate
    }
    recession_df.append(metrics)

recession_df = pd.DataFrame(recession_df)
recession_df

In [ ]:
non_recession_df = []

for asset in asset_list:
    df = data[asset]
    mean_non_rec_strategy_total_returns = df["nonrec_strategy total return"].mean()
    median_non_rec_strategy_total_returns = df["nonrec_strategy total return"].median()
    mean_non_rec_strategy_annualized_return = df["nonrec_ann. strategy return"].mean()
    median_non_rec_strategy_annualized_return = df["nonrec_ann. strategy return"].median()
    mean_non_rec_strategy_volatility = df["nonrec_ann. strategy volatility"].mean()
    median_non_rec_strategy_volatility = df["nonrec_ann. strategy volatility"].median()
    mean_non_rec_strategy_sharpe = df["nonrec_strategy sharpe ratio"].mean()
    median_non_rec_strategy_sharpe = df["nonrec_strategy sharpe ratio"].median()
    mean_non_rec_strategy_winrate = df["nonrec_strategy winrate"].mean()
    median_non_rec_strategy_winrate = df["nonrec_strategy winrate"].median()

    metrics = {
        "asset": asset,
        "mean_non_rec_strategy_total_returns": mean_non_rec_strategy_total_returns,
        "median_non_rec_strategy_total_returns": median_non_rec_strategy_total_returns,
        "mean_non_rec_strategy_annualized_return": mean_non_rec_strategy_annualized_return,
        "median_non_rec_strategy_annualized_return": median_non_rec_strategy_annualized_return,
        "mean_non_rec_strategy_volatility": mean_non_rec_strategy_volatility,
        "median_non_rec_strategy_volatility": median_non_rec_strategy_volatility,
        "mean_non_rec_strategy_sharpe": mean_non_rec_strategy_sharpe,
        "median_non_rec_strategy_sharpe": median_non_rec_strategy_sharpe,
        "mean_non_rec_strategy_winrate": mean_non_rec_strategy_winrate,
        "median_non_rec_strategy_winrate": median_non_rec_strategy_winrate
    }
    non_recession_df.append(metrics)

non_recession_df = pd.DataFrame(non_recession_df)
non_recession_df

In [ ]:
strategy = "{strategy_name}"
performance_df.to_excel(f"performance_summary_{strategy}.xlsx", index=False)
significance_df.to_excel(f"significance_summary_{strategy}.xlsx", index=False)
significance_counts_df.to_excel(f"significance_counts_summary_{strategy}.xlsx", index=False)
high_vol_df.to_excel(f"high_vol_summary_{strategy}.xlsx", index=False)
low_vol_df.to_excel(f"low_vol_summary_{strategy}.xlsx", index=False)
recession_df.to_excel(f"recession_summary_{strategy}.xlsx", index=False)
non_recession_df.to_excel(f"non_recession_summary_{strategy}.xlsx", index=False)

# high_vol_buy_n_hold_df.to_excel(f"high_vol_buy_n_hold_summary_{strategy}.xlsx", index=False)
# low_vol_buy_n_hold_df.to_excel(f"low_vol_buy_n_hold_summary_{strategy}.xlsx", index=False)